In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.callbacks import TensorBoard
#from sklearn.externals import joblib
import datetime
from keras.models import model_from_json

import numpy as np
import pandas as pd
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle

import pickle

Using TensorFlow backend.


Loading Data and stitching together, to make preprocessing easier

In [6]:
data = pd.read_csv("Test.csv", usecols=['Topic','Sentiment', 'TweetText']).append(pd.read_csv("Train.csv", usecols=['Topic','Sentiment', 'TweetText']))

In [7]:
data.Topic.value_counts()

apple        1079
twitter       953
google        867
microsoft     856
Name: Topic, dtype: int64

In [8]:
num_of_categories = 850
shuffled = data.reindex(np.random.permutation(data.index))
a = shuffled[shuffled['Topic'] == 'apple'][:num_of_categories]
t = shuffled[shuffled['Topic'] == 'twitter'][:num_of_categories]
g = shuffled[shuffled['Topic'] == 'google'][:num_of_categories]
m = shuffled[shuffled['Topic'] == 'microsoft'][:num_of_categories]
concated = pd.concat([a,t,g,m], ignore_index=True)
#Shuffle the dataset
concated = shuffle(concated)
concated['LABEL'] = 0
concated.loc[concated['Topic'] == 'apple', 'LABEL'] = 0
concated.loc[concated['Topic'] == 'twitter', 'LABEL'] = 1
concated.loc[concated['Topic'] == 'google', 'LABEL'] = 2
concated.loc[concated['Topic'] == 'microsoft', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'Topic' in concated.keys():
    concated.drop(['Topic'], axis=1)
'''
 [1. 0. 0. 0.] a
 [0. 1. 0. 0.] t
 [0. 0. 1. 0.] g
 [0. 0. 0. 1.] m
'''

764     0
2392    2
3119    3
172     0
878     1
2753    3
2004    2
498     0
2195    2
2841    3
Name: LABEL, dtype: int64
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


'\n [1. 0. 0. 0.] a\n [0. 1. 0. 0.] t\n [0. 0. 1. 0.] g\n [0. 0. 0. 1.] m\n'

In [9]:
data.Sentiment.value_counts()

neutral       1631
irrelevant    1246
negative       475
positive       403
Name: Sentiment, dtype: int64

In [10]:
num_of_categories2 = 400
shuffled2 = data.reindex(np.random.permutation(data.index))
n = shuffled2[shuffled2['Sentiment'] == 'neutral'][:num_of_categories2]
i = shuffled2[shuffled2['Sentiment'] == 'irrelevant'][:num_of_categories2]
neg = shuffled2[shuffled2['Sentiment'] == 'negative'][:num_of_categories2]
pos = shuffled2[shuffled2['Sentiment'] == 'positive'][:num_of_categories2]
concated2 = pd.concat([n,i,neg,pos], ignore_index=True)
#Shuffle the dataset
concated2 = shuffle(concated2)
concated2['LABEL'] = 0
concated2.loc[concated2['Sentiment'] == 'neutral', 'LABEL'] = 0
concated2.loc[concated2['Sentiment'] == 'irrelevant', 'LABEL'] = 1
concated2.loc[concated2['Sentiment'] == 'negative', 'LABEL'] = 2
concated2.loc[concated2['Sentiment'] == 'positive', 'LABEL'] = 3
print(concated2['LABEL'][:10])
labels2 = to_categorical(concated2['LABEL'], num_classes=4)
print(labels2[:10])
if 'Sentiment' in concated2.keys():
    concated2.drop(['Sentiment'], axis=1)
'''
 [1. 0. 0. 0.] n
 [0. 1. 0. 0.] i
 [0. 0. 1. 0.] neg
 [0. 0. 0. 1.] pos
'''

885     2
1477    3
1270    3
832     2
1135    2
275     0
1595    3
104     0
1361    3
982     2
Name: LABEL, dtype: int64
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


'\n [1. 0. 0. 0.] n\n [0. 1. 0. 0.] i\n [0. 0. 1. 0.] neg\n [0. 0. 0. 1.] pos\n'

In [11]:
n_most_common_words = 500
max_len = 130
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['TweetText'].values)
sequences = tokenizer.texts_to_sequences(concated['TweetText'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 9338 unique tokens.


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [13]:
print(X_test.shape, X_train.shape)
print(y_test.shape, y_train.shape)

(850, 130) (2550, 130)
(850, 4) (2550, 4)


In [14]:
tokenizer2 = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer2.fit_on_texts(concated2['TweetText'].values)
sequences2 = tokenizer2.texts_to_sequences(concated2['TweetText'].values)
word_index2 = tokenizer2.word_index
print('Found %s unique tokens.' % len(word_index2))

X2 = pad_sequences(sequences2, maxlen=max_len)

Found 5452 unique tokens.


In [15]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2 , labels2, test_size=0.25, random_state=42)

In [16]:
print(X_test2.shape, X_train2.shape)
print(y_test2.shape, y_train2.shape)

(400, 130) (1200, 130)
(400, 4) (1200, 4)


In [24]:
def create_model(x, y, x_val, y_val, embed_dim = 128, lstm = 64, epochs = 10, batch_size = 256, optimizer='adam', verbose=1, name="lstm_"):
    model = Sequential()
    model.add(Embedding(500, embed_dim, input_length=x.shape[1]))
    model.add(SpatialDropout1D(0.7))
    model.add(LSTM(lstm, dropout=0.7, recurrent_dropout=0.7))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    print(model.summary())
    filename = "ed:{},lstm:{},ep:{},bs:{},opt:{},ts:{}".format(embed_dim, lstm, epochs, batch_size, optimizer, datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    history = model.fit(x, y, epochs=epochs, verbose=verbose, callbacks=[TensorBoard(log_dir="tb/" + name + filename, histogram_freq=0, write_graph=False)], 
                        validation_data=(x_val, y_val), batch_size=batch_size)
    with open("models/" + name + "model_" + filename + ".json", "w") as json_file:
        json_file.write(model.to_json())
    model.save_weights("models/" + name + "model_" + filename + ".h5")
    return model

Hyperparameters used for Gridsearch

In [18]:
"""epochs = [10, 20, 30] #22
batch_size = [150, 250, 500] #150

embed_dims = [50, 100, 200]  #standard
lstm = [30, 50, 100]  #standard
optimizer = ['Nadam', 'Adadelta', 'Adagrad', 'Adam', 'RMSprop']  #adagrad"""

"epochs = [10, 20, 30] #22\nbatch_size = [150, 250, 500] #150\n\nembed_dims = [50, 100, 200]  #standard\nlstm = [30, 50, 100]  #standard\noptimizer = ['Nadam', 'Adadelta', 'Adagrad', 'Adam', 'RMSprop']  #adagrad"

In [25]:
cat_model = create_model(X_train, y_train, X_test, y_test, epochs=22, batch_size=150, optimizer='Adagrad', name="cat")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 130, 128)          64000     
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 113,668
Trainable params: 113,668
Non-trainable params: 0
_________________________________________________________________
None
Train on 2550 samples, validate on 850 samples
Epoch 1/22
2550/2550 [==============================] - 3s 1ms/step - loss: 1.3332 - acc: 0.3753 - val_loss: 1.2072 - val_acc: 0.5624
Epoch 2/22
2550/2550 [==============================] - 3s 1ms/step - loss: 1.1559

In [26]:
sent_model = create_model(X_train2, y_train2, X_test2, y_test2, epochs=22, batch_size=150, optimizer='Adagrad', name="sent")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 130, 128)          64000     
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 260       
Total params: 113,668
Trainable params: 113,668
Non-trainable params: 0
_________________________________________________________________
None
Train on 1200 samples, validate on 400 samples
Epoch 1/22
1200/1200 [==============================] - 1s 1ms/step - loss: 1.3759 - acc: 0.3075 - val_loss: 1.3442 - val_acc: 0.4525
Epoch 2/22
1200/1200 [==============================] - 1s 1ms/step - loss: 1.3271

In [27]:
accr = cat_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

850/850 [==============================] - 1s 1ms/step
Test set
  Loss: 0.521
  Accuracy: 0.795


In [28]:
accr2 = sent_model.evaluate(X_test2,y_test2)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr2[0],accr2[1]))

400/400 [==============================] - 0s 1ms/step
Test set
  Loss: 1.001
  Accuracy: 0.588


In [29]:
txt = ["Getting a lot of spam at my iphone thats nice"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = cat_model.predict(padded)
labels = ['apple', 'twitter', 'google', 'microsoft']
print(pred, labels[np.argmax(pred)])
seq = tokenizer2.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = sent_model.predict(padded)
labels = ['neutral', 'irrelevant', 'negative', 'positive']
print(pred, labels[np.argmax(pred)])

[[0.38740495 0.13121562 0.2631111  0.21826828]] apple
[[0.13929802 0.00575931 0.33990443 0.5150383 ]] positive


In [32]:
with open('models/sent_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer2, handle, protocol=pickle.HIGHEST_PROTOCOL)